# Возможности применения применения конструкции match/case

In [75]:
import random

## Базовый синтаксис

In [76]:
def handle_product(status):
    match status:
        case "ordered":
            return "Товар заказан."
        case "delivering":
            return "Товар едет в пункт выдачи."
        case "arrived":
            return "Товар прибыл в пункт выдачи товаров."
        case _:
            return "Ошибка статуса товара."

In [77]:
# product_status = input("Введите статус товара:")
product_status = "arrived"
print(handle_product(product_status))

Товар прибыл в пункт выдачи товаров.


## Паттерн-матчинг

### Работа со списками любой длины

In [ ]:
def process_items(items):
    match items:
        case []:
            print("Пустой список")
        case [first]:
            print(f"Один элемент: {first}")
        case [first, second]:
            print(f"Два элемента: {first} и {second}")
        case [first, *rest]:
            print(f"Первый: {first}, остальные: {rest}")

process_items([1, 2, 3, 4])  # Первый: 1, остальные: [2, 3, 4]

### Распаковка структур прямо в case

In [78]:
class Point:
    __match_args__ = ("x", "y")
    
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return f"Point({self.x}, {self.y})"

In [79]:
def random_point(x_min, x_max, y_min, y_max):
    x = random.randint(x_min, x_max)
    y = random.randint(y_min, y_max)
    return Point(x, y)

def where_is(point):
    match point:
        case [point_1, *rest]:
            print(f"Первый элемент списка: {point_1}")
        case Point(0, 0):
            print("Точка в начале координат.")
        case Point(x, 0):
            print(f"Точка на оси X с координатой x={x}.")
        case Point(0, y):
            print(f"Точка на оси Y с координатой y={y}.")
        case Point (x, y):
            print(f"Точка в координатах x={x}, y={y}.")
        case _:
            print("Это не точка")

In [80]:
point_list = [Point(x, y) for x,y in [(0, 0), (6, 0), (0, 7), (2, 3), (4, 4)]]

In [81]:
for point in point_list:
    where_is(point)
where_is(point_list)

Точка в начале координат.
Точка на оси X с координатой x=6.
Точка на оси Y с координатой y=7.
Точка в координатах x=2, y=3.
Точка в координатах x=4, y=4.
Первый элемент списка: Point(0, 0)


### Условия внутри case (Guards)

In [ ]:
def check_point(point):
    match point:
        case (x, y) if x == y:
            print(f"Точка на диагонали: ({x}, {y})")
        case (x, y) if x > 0 and y > 0:
            print(f"Точка в первом квадранте: ({x}, {y})")
        case (x, y):
            print(f"Просто точка: ({x}, {y})")

check_point((5, 5))  # Точка на диагонали: (5, 5)
check_point((3, 4))  # Точка в первом квадранте: (3, 4)

### Сопоставление по типу

In [ ]:
def process_value(value):
    match value:
        case int():
            print(f"Целое число: {value}")
        case float():
            print(f"Дробное число: {value}")
        case str() as text if len(text) > 10:
            print(f"Длинная строка: {text[:10]}...")
        case str() as text:
            print(f"Строка: {text}")
        case list() | tuple() as sequence:
            print(f"Последовательность длиной {len(sequence)}")
        case _:
            print("Что-то другое")

process_value(42)           # Целое число: 42
process_value([1, 2, 3])    # Последовательность длиной 3

### Работа с классами и dataclass

In [ ]:
from dataclasses import dataclass
from typing import Literal

@dataclass
class User:
    name: str
    role: Literal["admin", "editor", "viewer"]
    active: bool

def handle_user(user: User):
    match user:
        case User(name="admin", role="admin"):
            print("Супер-администратор")
        case User(name=name, role="admin"):
            print(f"Администратор {name}")
        case User(role="editor", active=True):
            print("Активный редактор")
        case User(role="viewer", active=False):
            print("Неактивный зритель")
        case User(name=name, active=True):
            print(f"Активный пользователь: {name}")
        case _:
            print("Неизвестный пользователь")

# Тестируем
admin = User(name="alice", role="admin", active=True)
editor = User(name="bob", role="editor", active=False)

handle_user(admin)   # Администратор alice
handle_user(editor)  # Активный пользователь: bob

## Примеры использования

### Обработка JSON-ответов API

In [ ]:
def process_data(data):
    print("Обработка данных...")

def process_object(data):
    print("Обработка объекта...")

In [ ]:
def handle_api_response(response):
    match response:
        case {"status": "success", "data": list(data)}:
            print(f"Успех! Получено {len(data)} элементов")
            return process_data(data)
        
        case {"status": "error", "code": 404, "message": msg}:
            print(f"Не найдено: {msg}")
            return None
        
        case {"status": "error", "code": int(code), "message": msg}:
            print(f"Ошибка {code}: {msg}")
            return None
        
        case {"status": "success", "data": dict(data)}:
            print(f"Успех! Получен объект")
            return process_object(data)
        
        case _:
            print(f"Неизвестный формат ответа: {response}")
            return None

### Обработка команд CLI

In [ ]:
def handle_command(command_line):
    match command_line.split():
        case ["exit"]:
            print("Выход из программы")
            return False
        
        case ["help"]:
            print("Доступные команды: help, exit, copy, move")
            return True
        
        case ["copy", src, dest]:
            print(f"Копируем {src} в {dest}")
            return True
        
        case ["move", src, dest] if src != dest:
            print(f"Перемещаем {src} в {dest}")
            return True
        
        case ["move", path]:
            print(f"Ошибка: исходный и целевой пути одинаковы")
            return True
        
        case ["search", *terms]:
            print(f"Поиск по терминам: {terms}")
            return True
        
        case _:
            print(f"Неизвестная команда: {command_line}")
            return True